<a href="https://colab.research.google.com/github/mpily/basic-ml-course/blob/Project/datamining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#Example from previous project you could use it to learn how to do the mining
# basic imports
import json
import urllib.request

################################ Data mining from the FPL API ################################
# The only 2 parameters to specify are "teamID" and "lastGameweek"
teamID = 3637734
lastGameweek = 10

# Get the detailed info about a given FPL Manager’s Team and a given game week
gameweekData = {}
for i in range(1, lastGameweek + 1):
    base = "https://fantasy.premierleague.com/api/entry/" + str(teamID) + "/event/" + str(i) + "/picks/"
    page = urllib.request.urlopen(base)
    data = {"GW" + str(i) : json.load(page)}
    gameweekData.update(data)

# Get the general info about a given FPL Manager’s Team
base = "https://fantasy.premierleague.com/api/entry/" + str(teamID) + "/"
page = urllib.request.urlopen(base)
dataTeamInfo = json.load(page)

# Get the generic info about PL teams, the players and the game week details
base = "https://fantasy.premierleague.com/api/bootstrap-static/" 
page = urllib.request.urlopen(base)
dataGeneral = json.load(page)
events = dataGeneral["events"]
elements = dataGeneral["elements"]
teams   = dataGeneral["teams"]

In [9]:
# Get the detailed info about a specific premier league player   
def getPlayerPointsAtOneGW(playerID, gameweek):
    base = "https://fantasy.premierleague.com/api/element-summary/" + str(playerID) + "/"
    page = urllib.request.urlopen(base)
    datagw = json.load(page)
    gwPoints = 0
    for i in range(len(datagw["history"])): 
        if gameweek == datagw["history"][i]["round"]: # to account for absent gameweek in the json
            gwPoints = gwPoints + datagw["history"][i]["total_points"] # to account for double gw            
    return gwPoints

def getPlayerName(playerID):
    i = 0
    while i < len(elements):
        if (elements[i]["id"] == playerID):
            return (elements[i]["first_name"] + " " + elements[i]["second_name"])
        i += 1
    return "ID not found"

positions = ["GK", "DEF", "MID", "ST"]
def getPlayerPosition(playerID):
    i = 0
    while i < len(elements):
        if (elements[i]["id"] == playerID):
                playersElementType = elements[i]["element_type"]
                playerPosition = positions[playersElementType - 1]
                return playerPosition
        i += 1
    return "ID not found"
print(elements[0].keys())

dict_keys(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code', 'cost_change_event', 'cost_change_event_fall', 'cost_change_start', 'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo', 'points_per_game', 'second_name', 'selected_by_percent', 'special', 'squad_number', 'status', 'team', 'team_code', 'total_points', 'transfers_in', 'transfers_in_event', 'transfers_out', 'transfers_out_event', 'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index', 'starts', 'expected_goals', 'expected_assists', 'expected_goal_involvements', 'expected_goals_conceded', 'influence_rank', 'influence_rank_type', 'creativity_rank', 'creativity_rank_ty

In [16]:
################################ Calculations ################################
# Get the specific team data from the json in organised lists and dictionaries
teamName = dataTeamInfo["name"]
points = []
gameweekRank = []
overallRank = []
teamValue = []
transfers = []
transfersCost = []
averagePoints = []
highestPoints = []
captain = []
captainPoints = []
startingTeam = {}
totalPointsPerLine = {}
totalPointsPerLineSeason = {"GK" : 0, "DEF" : 0, "MID" : 0, "ST" : 0}

for gw in range(1, lastGameweek + 1):
    # List with basic data each gw

    points.append(gameweekData["GW" + str(gw)]["entry_history"]["points"])
    gameweekRank.append(gameweekData["GW" + str(gw)]["entry_history"]["rank"])
    overallRank.append(gameweekData["GW" + str(gw)]["entry_history"]["overall_rank"])
    teamValue.append(gameweekData["GW" + str(gw)]["entry_history"]["value"])
    transfers.append(gameweekData["GW" + str(gw)]["entry_history"]["event_transfers"])
    transfersCost.append(gameweekData["GW" + str(gw)]["entry_history"]["event_transfers_cost"])
    averagePoints.append(events[gw-1]["average_entry_score"])
    highestPoints.append(events[gw-1]["highest_score"])
    
    # Dict with starting team each gw
    startingTeam["GW" + str(gw)] = {}
    for j in range(0, 15):
        if gameweekData["GW" + str(gw)]["picks"][j]["is_captain"] == True:
            captain.append(getPlayerName(gameweekData["GW" + str(gw)]["picks"][j]["element"]))
            captainPoints.append(getPlayerPointsAtOneGW(gameweekData["GW" + str(gw)]["picks"][j]["element"], gw))
    for n in range(0, 15):
        startingTeam["GW" + str(gw)]["player" + str(n)] = {}
        startingTeam["GW" + str(gw)]["player" + str(n)]["name"] = getPlayerName(gameweekData["GW" + str(gw)]["picks"][n]["element"])
        startingTeam["GW" + str(gw)]["player" + str(n)]["position"] = getPlayerPosition(gameweekData["GW" + str(gw)]["picks"][n]["element"])
        startingTeam["GW" + str(gw)]["player" + str(n)]["points"] = getPlayerPointsAtOneGW(gameweekData["GW" + str(gw)]["picks"][n]["element"], gw)

    # Dict with points per line each gw
    totalPointsPerLine["GW" + str(gw)] = {"GK" : 0, "DEF" : 0, "MID" : 0, "ST" : 0}
    for player in range(0, 11): # que les titulaires de 0 à 11 donc
        if startingTeam["GW" + str(gw)]["player" + str(player)]["position"] == "GK":
            totalPointsPerLine["GW" + str(gw)]["GK"] = totalPointsPerLine["GW" + str(gw)]["GK"] + startingTeam["GW" + str(gw)]["player" + str(player)]["points"]
        elif startingTeam["GW" + str(gw)]["player" + str(player)]["position"] == "DEF":
            totalPointsPerLine["GW" + str(gw)]["DEF"] = totalPointsPerLine["GW" + str(gw)]["DEF"] + startingTeam["GW" + str(gw)]["player" + str(player)]["points"]
        elif startingTeam["GW" + str(gw)]["player" + str(player)]["position"] == "MID":
            totalPointsPerLine["GW" + str(gw)]["MID"] = totalPointsPerLine["GW" + str(gw)]["MID"] + startingTeam["GW" + str(gw)]["player" + str(player)]["points"]
        elif startingTeam["GW" + str(gw)]["player" + str(player)]["position"] == "ST":
            totalPointsPerLine["GW" + str(gw)]["ST"] = totalPointsPerLine["GW" + str(gw)]["ST"] + startingTeam["GW" + str(gw)]["player" + str(player)]["points"]
    # Dict with points per line for the entire season       
    totalPointsPerLineSeason["GK"] = totalPointsPerLineSeason["GK"] + totalPointsPerLine["GW" + str(gw)]["GK"]
    totalPointsPerLineSeason["DEF"] = totalPointsPerLineSeason["DEF"] + totalPointsPerLine["GW" + str(gw)]["DEF"]
    totalPointsPerLineSeason["MID"] = totalPointsPerLineSeason["MID"] + totalPointsPerLine["GW" + str(gw)]["MID"]
    totalPointsPerLineSeason["ST"] = totalPointsPerLineSeason["ST"] + totalPointsPerLine["GW" + str(gw)]["ST"]
    
    print("GW" + str(gw) + " : Done.")

GW1 : Done.
GW2 : Done.
GW3 : Done.
GW4 : Done.
GW5 : Done.
GW6 : Done.
GW7 : Done.
GW8 : Done.
GW9 : Done.
GW10 : Done.


In [ ]:
#Player History
import pandas as pd
url = 'https://raw.githubusercontent.com/mpily/basic-ml-course/master/fplAnalytics-playerStautsData.csv'
player_data = pd.read_csv(url)
class playerStatus():
  def __init__(self,minutes,total_points,bonus,points_per_game,selected_by_percent):
    self.time_played = minutes
    self.total_points = total_points
    self.bonus_gained = bonus
    self.avg_points_per_game = points_per_game
    self.avg_selected_by_percent = selected_by_percent

player_data_dict = {}

for idx,row in player_data.iterrows():
  player_data_dict[row['name']] = playerStatus(row['minutes'],row['total_points'],row['bonus'],
                                               row['points_per_game'],row['selected_by_percent'])



Elements contains a lot of player data. It can be viewed with


```
print(elements[0].keys())
```
For every week data is mostly going to be the same for each player but we will have to add points of previous week so as to create the difference


In [26]:
#####################Playing Against team######################################
base = "https://fixturedownload.com/feed/json/epl-2022"
page = urllib.request.urlopen(base)
fixtures = json.load(page)
def getOpponents(team, gameweek):
    if team == 'Nott\'m Forest':
      team = 'Nottingham Forest'
    for i in range(len(fixtures)): 
        if fixtures[i]["RoundNumber"] != gameweek:
          continue
        if team == fixtures[i]["HomeTeam"]:
          return fixtures[i]["AwayTeam"]
        elif team == fixtures[i]["AwayTeam"]:
          return fixtures[i]["HomeTeam"]
    print(team, gameweek)
    assert False, "Team doesn't exist or game week doesn't exist"

In [44]:
import numpy as np
print(teams[0].keys())
element_parameters = []
bad_parameters     = ['code','id','first_name','second_name','photo','total_points']
for param in elements[0].keys():
  if param in bad_parameters:
    continue
  else :
    element_parameters.append(param)
num_of_gameweeks = 10
inputs           = []
results          = []
for i in range(len(elements)):
  for week in range(1,num_of_gameweeks+1):
    next_input = []
    for param in element_parameters:
      next_input.append(elements[i][param])
    next_input.append(getOpponents(teams[elements[i]['team']-1]['name'],week))
    inputs.append(next_input)
    results.append(elements[i]['total_points'])
element_parameters.append("opponent_team")
inputs_numpy = np.array(inputs)
results_numpy = np.array(results)
results_numpy.shape = [len(results),1]
print(results_numpy.shape)
print(inputs_numpy.shape)

dict_keys(['code', 'draw', 'form', 'id', 'loss', 'name', 'played', 'points', 'position', 'short_name', 'strength', 'team_division', 'unavailable', 'win', 'strength_overall_home', 'strength_overall_away', 'strength_attack_home', 'strength_attack_away', 'strength_defence_home', 'strength_defence_away', 'pulse_id'])
(6670, 1)
(6670, 83)


In [45]:
print(inputs_numpy, results_numpy)
combined_inputs_outputs = np.append(inputs_numpy,results_numpy,axis=1)
print(combined_inputs_outputs.shape)
element_parameters.append("points")
df = pd.DataFrame(combined_inputs_outputs, columns = element_parameters)
df.to_csv('data_fpl.csv')

[[100 100 0 ... 0.0 0.0 'Crystal Palace']
 [100 100 0 ... 0.0 0.0 'Leicester']
 [100 100 0 ... 0.0 0.0 'Bournemouth']
 ...
 [None None 0 ... 0.0 0.0 'Man City']
 [None None 0 ... 0.0 0.0 'West Ham']
 [None None 0 ... 0.0 0.0 'Chelsea']] [[2]
 [2]
 [2]
 ...
 [1]
 [1]
 [1]]
(6670, 84)


In [47]:
from google.colab import files
files.download('data_fpl.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>